In [1]:
import pandas as pd
import numpy as np
import scipy.stats as stats

import matplotlib
import matplotlib.pyplot as plt

import plotly
import plotly.graph_objects as go

import ipywidgets as widgets

In [2]:
CARB_TYPES = {
    "FAST_CARB": {"peak": 15, "duration": 50, "onset": 0},
    "SIMPLE_CARB": {"peak": 30, "duration": 200, "onset": 0},
    "COMPLEX_CARB": {"peak": 60, "duration": 300, "onset": 0},
    "FATTY_CARB": {"peak": 90, "duration": 300, "onset": 0}
}

INSULIN_TYPES = {
    "RAPID": {"peak": 80, "duration": 300, "onset": 15}
}

MEAL_COLOR = 'sandybrown'
BOLUS_COLOR = 'lightblue'

CARB_LINE = "peru"
INSULIN_LINE = "steelblue"

CARB_AREA = "navajowhite"
INSULIN_AREA = "lightcyan"

#how many units of glucose per unit carb/insulin
CARB_COEF = .1
INSULIN_COEF = 1

MAX_CARBS = 9000
MAX_INSULIN = 9000

BASE_GLUCOSE = 0
#desired glucose range is base glucose plus or minus this many units
GLUCOSE_TOLERANCE = 20

#scales this y axis max/min to this many times the max/min glucose response
Y_AXIS_SCALE = 3

In [3]:
class GammaDistribution:
    def __init__(self, time, peak, duration, onset = 0):
        self.peak = peak
        self.duration = duration
        self.onset = onset
        
        self.calculate(time)
        
    def calculate(self, time):
        self.start_time = time + self.onset
        self.end_time = time + self.onset + self.duration
        
        self.t = np.linspace(self.start_time, self.end_time, self.duration + 1)
        
        #calculate gamma distribution parameters, very hacky
        self.scale = self.duration / 10
        self.a = 10 - sum(
            [2 ** (2 - p) for p in range(round(6 * self.scale / self.peak))]
        )
        
        """turn into properties"""   
        self._pdf = stats.gamma.pdf(
            self.t, 
            a = self.a, 
            loc = self.start_time, 
            scale = self.scale
        )
        self._cdf = stats.gamma.cdf(
            self.t, 
            a = self.a, 
            loc = self.start_time, 
            scale = self.scale
        )
        
    def pdf(self, t = None):
        #returns the probability density function or its value at a specified point in time
        if t is None:
            return self._pdf
        else:
            t = round(t)
            if t < self.start_time:
                return 0
            if t > self.end_time:
                return 0
            else:
                return self._pdf[t - self.start_time]
        
    def cdf(self, t = None):
        #returns the cumulative distribution function or its value at a specified point in time
        if t is None:
            return self._cdf
        else:
            t = round(t)
            if t < self.start_time:
                return 0
            if t > self.end_time:
                return self._cdf[-1]
            else:
                return self._cdf[t - self.start_time]

In [4]:
class Bolus:
    def __init__(self, parent, time, dose, insulin_type = "RAPID"):
        self.parent = parent
        
        self.time = time
        self.dose = dose
        self.insulin_type = insulin_type
        
        self.data = GammaDistribution(self.time, **INSULIN_TYPES[self.insulin_type])
        
        self.gui()
        
    def insulin(self, cumulative = False, t = None):
        if cumulative:
            return self.data.cdf(t = t) * self.dose
        else:
            return self.data.pdf(t = t) * self.dose
    
    def glucose(self, cumulative = True, t = None):
        #insulin produces a glucose decrease
        return self.insulin(cumulative = cumulative, t = t) * -INSULIN_COEF
    
    def gui(self):
        #structure the containers
        self.insulin_type_dropdown = widgets.Dropdown(
            options = INSULIN_TYPES.keys(), 
            value = self.insulin_type, 
            description = "Type:"
        )
        self.dose_field = widgets.BoundedIntText(
            value = self.dose, 
            min = 0, 
            max = MAX_INSULIN, 
            description = "Dose:"
        )
        self.time_label = widgets.Label()
        self.details_row = widgets.HBox([
            self.insulin_type_dropdown, 
            self.dose_field, 
            self.time_label
        ], layout = widgets.Layout(
            justify_content = "space-between"
        ))
        
        self.t_min_label = widgets.Label(str(self.parent.t_min))
        self.time_slider = widgets.IntSlider(
            value = self.time, 
            min = self.parent.t_min, 
            max = self.parent.t_max, 
            readout = False, 
            layout = widgets.Layout(
                width = '95%'
            )
        )
        self.t_max_label = widgets.Label(str(self.parent.t_max))
        self.timeline_row = widgets.HBox([
            self.t_min_label, 
            self.time_slider, 
            self.t_max_label
        ], layout = widgets.Layout(
            justify_content = "space-between"
        ))
        
        self.widgets = widgets.VBox([
            self.details_row, 
            self.timeline_row
        ], layout = {
            'border':'1px solid black',
            'padding':'4px'
        })
        
        #add interactivity
        widgets.dlink(
            (self.time_slider, 'value'), 
            (self.time_label, 'value'), 
            lambda x: "Time: " + str(x)
        )
        widgets.dlink(
            (self.time_slider, 'min'),
            (self.t_min_label, 'value'), 
            lambda x: str(x)
        )
        widgets.dlink(
            (self.time_slider, 'max'),
            (self.t_max_label, 'value'), 
            lambda x: str(x)
        )
        self.insulin_type_dropdown.observe(self.update_data, names='value')
        self.dose_field.observe(self.update_data, names='value')
        self.time_slider.observe(self.update_data, names='value')
        
        return self.widgets
    
    def update_gui(self):
        self.time_slider.max = self.parent.t_max
        self.time_slider.min = self.parent.t_min
    
    def update_data(self, e):
        self.insulin_type = self.insulin_type_dropdown.value
        self.dose = self.dose_field.value
        self.time = self.time_slider.value
        
        self.data = GammaDistribution(self.time, **INSULIN_TYPES[self.insulin_type])
        
        self.parent.update()
        
    def update_traces(self):
        self.line = go.Scatter(
            x = self.data.t, 
            y = self.parent.y_max - self.insulin() * 30, 
            line = dict(color = BOLUS_COLOR),
            mode = 'lines', 
            name = self.insulin_type
        )
        self.base = go.Scatter(
            x = self.data.t,
            y = [self.parent.y_max for t in self.data.t],
            mode = 'lines',
            line = dict(color = BOLUS_COLOR),
            fill = 'tonexty'
        )
        
        self.traces = [
            self.line, 
            self.base
        ]
        return self.traces

In [5]:
class Meal:
    def __init__(self, parent, time, carb_quantity, carb_type):
        self.parent = parent
        
        self.time = time
        self.carb_quantity = carb_quantity
        self.carb_type = carb_type
                
        self.data = GammaDistribution(self.time, **CARB_TYPES[self.carb_type])
        
        self.gui()
        
    def carbs(self, cumulative = False, t = None):
        if cumulative:
            return self.data.cdf(t = t) * self.carb_quantity
        else:
            return self.data.pdf(t = t) * self.carb_quantity
    
    def glucose(self, cumulative = True, t = None):
        #carbs produce a glucose increase
        return self.carbs(cumulative = cumulative, t = t) * CARB_COEF
    
    def gui(self):
        #structure the containers
        self.carb_type_dropdown = widgets.Dropdown(
            options = CARB_TYPES.keys(), 
            value = self.carb_type, 
            description = "Type:"
        )
        self.carb_quantity_field = widgets.BoundedIntText(
            value = self.carb_quantity, 
            min = 0, 
            max = MAX_CARBS, 
            description = "Carbs:"
        )
        self.time_label = widgets.Label()
        self.details_row = widgets.HBox([
            self.carb_type_dropdown, 
            self.carb_quantity_field, 
            self.time_label
        ], layout = widgets.Layout(
            justify_content = "space-between"
        ))
        
        self.t_min_label = widgets.Label(str(self.parent.t_min))
        self.time_slider = widgets.IntSlider(
            value = self.time, 
            min = self.parent.t_min, 
            max = self.parent.t_max, 
            readout = False, 
            layout = widgets.Layout(
                width = '95%'
            )
        )
        self.t_max_label = widgets.Label(str(self.parent.t_max))
        self.timeline_row = widgets.HBox([
            self.t_min_label, 
            self.time_slider,
            self.t_max_label
        ], layout = widgets.Layout(
            justify_content = "space-between"
        ))
               
        self.widgets = widgets.VBox([
            self.timeline_row, 
            self.details_row
        ], layout = {
            'border':'1px solid black',
            'padding':'8px'
        })
        
        #add interactivity
        widgets.dlink(
            (self.time_slider, 'value'),
            (self.time_label, 'value'), 
            lambda x: "Time: " + str(x)
        )
        widgets.dlink(
            (self.time_slider, 'min'),
            (self.t_min_label, 'value'), 
            lambda x: str(x)
        )
        widgets.dlink(
            (self.time_slider, 'max'),
            (self.t_max_label, 'value'), 
            lambda x: str(x)
        )
        self.carb_type_dropdown.observe(self.update_data, names='value')
        self.carb_quantity_field.observe(self.update_data, names='value')
        self.time_slider.observe(self.update_data, names='value')
        
        return self.widgets
    
    def update_gui(self):
        self.time_slider.max = self.parent.t_max
        self.time_slider.min = self.parent.t_min
    
    def update_data(self, e):
        self.carb_type = self.carb_type_dropdown.value
        self.carb_quantity = self.carb_quantity_field.value
        self.time = self.time_slider.value
        
        self.data = GammaDistribution(self.time, **CARB_TYPES[self.carb_type])
        
        self.parent.update()
        
    def update_traces(self):
        self.base = go.Scatter(
            x = self.data.t,
            y = [self.parent.y_min for t in self.data.t],
            mode = 'lines',
            line = dict(color = MEAL_COLOR)
        )
        self.line = go.Scatter(
            x = self.data.t, 
            y = self.carbs() * 3 + self.parent.y_min, 
            mode = 'lines',
            line = dict(color = MEAL_COLOR),
            fill = 'tonexty',
            name = self.carb_type
        )
        
        self.traces = [
            self.base, 
            self.line
        ]
        return self.traces

In [6]:
class Insulearn_Vis:
    def __init__(self):
        self.plot_gammas = True
        
        self.plot_response_effects = True
        
        self.base_glucose = BASE_GLUCOSE
        self.glucose_tolerance = GLUCOSE_TOLERANCE
        
        self.t_min = 0
        self.t_max = 480
        self.t_interval = 60
        self.t = np.linspace(self.t_min, self.t_max, self.t_max - self.t_min + 1)
        self.t_tickvals = list(range(self.t_min, self.t_max, self.t_interval)) + [self.t_max]
        
        self.y_min = -Y_AXIS_SCALE * GLUCOSE_TOLERANCE
        self.y_max = Y_AXIS_SCALE * GLUCOSE_TOLERANCE

        self.meals = []
        self.boluses = []
        
        self.t_indices = {}
        
        self.gui()
        
        self.vis()
        
        self.update()
        
        display(self.widgets)
        
    def gui(self):
        self.bolus_section = widgets.VBox()
        self.vis_section = widgets.VBox(layout = {
            'border':'1px solid black',
            'padding':'4px'
        })
        self.meal_section = widgets.VBox()
        self.widgets = widgets.VBox([
            self.bolus_section, 
            self.vis_section, 
            self.meal_section
        ])
        return self.widgets
        
    def vis(self):                      
        layout = go.Layout(
            plot_bgcolor= 'rgba(0,0,0,0)',
            xaxis = dict(
                title = 'Time (min)', 
                gridcolor = 'lightgrey', 
                range = [self.t_min, self.t_max],
                fixedrange = True,
                tickvals = self.t_tickvals
            ),
            yaxis = dict(
                title = 'Glucose Response', 
                showticklabels = False, 
                range = [self.y_min, self.y_max],
                fixedrange = True
            ),
            showlegend = False,
            margin = go.layout.Margin(l = 0, r = 32, b = 0, t = 0)
        )
        
        self.fig = go.Figure(layout = layout)
        
        #desired glucose range
        self.fig.add_hrect(
            y0 = -self.glucose_tolerance, 
            y1 = self.glucose_tolerance, 
            line_width = 0, 
            fillcolor = "lightgrey", 
            opacity = 0.33
        )
        
        config = {'displayModeBar': False, 'staticPlot': True}
        
        self.fig_widget = go.FigureWidget(self.fig)
        self.vis_section.children += (self.fig_widget,)
        
    def add_bolus(self, *args, **kwargs):
        b = Bolus(self, *args, **kwargs)
        
        self.boluses.append(b)
        self.bolus_section.children += (b.widgets,)
        
        self.update()
    
    def add_meal(self, *args, **kwargs):
        m = Meal(self, *args, **kwargs)
        
        self.meals.append(m)
        self.meal_section.children += (m.widgets,)
        
        self.update()

    def update_t_axis(self):
        #update time axis range
        for m in self.meals:
            self.t_max = max(self.t_max, m.data.end_time)
        for b in self.boluses:
            self.t_max = max(self.t_max, b.data.end_time)
        self.t = np.linspace(self.t_min, self.t_max, self.t_max - self.t_min + 1)
        for m in self.meals:
            m.update_gui()
        for b in self.boluses:
            b.update_gui()
        
    def update_data(self):
        self.carb_response = []
        self.insulin_response = []
        self.overall_response = []
        
        for t in self.t:
            #instantaneous glucose response to carbs at time t
            i_response_carbs = sum(
                [m.glucose(cumulative = False, t = t) for m in self.meals]
            )
            #cumulative glucose response to carbs at time t
            c_response_carbs = sum(
                [m.glucose(cumulative = True, t = t) for m in self.meals]
            )
            #instantaneous glucose response to insulin at time t
            i_response_insulin = sum(
                [b.glucose(cumulative = False, t = t) for b in self.boluses]
            )
            #cumulative glucose response to insulin at time t
            c_response_insulin = sum(
                [b.glucose(cumulative = True, t = t) for b in self.boluses]
            )
            
            self.carb_response.append(self.base_glucose + c_response_carbs)
            self.insulin_response.append(self.base_glucose + c_response_insulin)
            self.overall_response.append(self.base_glucose + c_response_carbs + c_response_insulin)
        
    def update_y_axis(self):    
        #update the y axis range
        if self.plot_response_effects:
            y_abs_max = max(
                max(self.carb_response), abs(min(self.insulin_response))
            )
            self.y_min = -y_abs_max
            self.y_max = y_abs_max
        else:
            y_abs_max = max(
                max(self.overall_response), abs(min(self.overall_response))
            )
            self.y_min = -Y_AXIS_SCALE * y_abs_max
            self.y_max = Y_AXIS_SCALE * y_abs_max
            
    def update_trace(self, name, new_trace):
        if name in self.t_indices:
            self.fig_widget.data[self.t_indices[name]].update(new_trace)
        else:
            self.t_indices[name] = len(self.fig_widget.data)
            self.fig_widget.add_trace(new_trace)
        
    def update_vis(self):
        with self.fig_widget.batch_update():
            #base glucose
            self.update_trace("base_glucose", go.Scatter(
                x = self.t, 
                y = [self.base_glucose for t in self.t], 
                mode ='lines',
                line = dict(color= 'grey', dash= 'dash'),
                name ='base'
            ))

            if self.plot_response_effects:
                #insulin glucose response
                if len(self.boluses) > 0:
                    self.update_trace("insulin_glucose" + "_a", go.Scatter(
                        x = self.t, 
                        y = self.insulin_response, 
                        mode = 'lines',
                        line = dict(
                            color = INSULIN_LINE,
                            width = 3
                        ),
                        name = 'glucose - carbs'
                    ))
                    self.update_trace("insulin_glucose" + "_b", go.Scatter(
                        x = self.t, 
                        y = self.overall_response, 
                        mode = 'lines',
                        line = dict(
                            color = CARB_AREA,
                            width = 0
                        ),
                        fill = 'tonexty'
                    ))

                #carb glucose response
                if len(self.meals) > 0:
                    self.update_trace("carb_glucose" + "_a", go.Scatter(
                        x = self.t, 
                        y = self.carb_response, 
                        mode = 'lines',
                        line = dict(
                            color = INSULIN_AREA,
                            width = 0
                        ),
                        fill = 'tonexty'
                    ))
                    self.update_trace("carb_glucose" + "_b", go.Scatter(
                        x = self.t, 
                        y = self.carb_response, 
                        mode = 'lines',
                        line = dict(
                            color = CARB_LINE,
                            width = 3
                        ),
                        name = 'glucose - insulin'
                    ))
                    
            #overall glucose response
            self.update_trace("overall_glucose", go.Scatter(
                x = self.t, 
                y = self.overall_response, 
                mode = 'lines',
                line = dict(
                    color = 'black', 
                    dash = 'dot', 
                    width = 5
                ),
                name = 'response'
            ))

            if self.plot_gammas:
                for i in range(len(self.boluses)):
                    self.update_trace("bolus_" + str(i) + "_a", self.boluses[i].update_traces()[0])
                    self.update_trace("bolus_" + str(i) + "_b", self.boluses[i].update_traces()[1])

                for i in range(len(self.meals)):
                    self.update_trace("meal_" + str(i) + "_a", self.meals[i].update_traces()[0])
                    self.update_trace("meal_" + str(i) + "_b", self.meals[i].update_traces()[1])
        
    def update(self):
        #self.update_t_axis()
        self.update_data()
        #self.update_y_axis()
        self.update_vis()

In [7]:
vis = Insulearn_Vis()
vis.add_meal(60, 300, "FAST_CARB")
vis.add_bolus(0, 50)
vis.add_meal(120, 200, "SIMPLE_CARB")

    'data': [{'line': {'color': 'grey', 'dash': 'dash'},
…